<a href="https://colab.research.google.com/github/dr-song-summer-project/ChatBot_Preprocessing/blob/main/refining_Data/Keyword_TF_IDF_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
!pip install xlrd
!pip install openpyxl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from math import log10

# =======================================
# -- TF-IDF function
# =======================================
def f(t, d):
    # d is document == tokens
    return d.count(t)

def tf(t, d):
    # d is document == tokens
    return 0.5 + 0.5*f(t,d)/max([f(w,d) for w in d])

def idf(t, D):
    # D is documents == document list
    numerator = len(D)
    denominator = 1 + len([ True for d in D if t in d])
    return log10(numerator/denominator)

def tfidf(t, d, D):
    return tf(t,d)*idf(t, D)

def tokenizer(d):
    # return [ t for t in d.split() if len(t) > 1 ]
    return d.split()

def tfidfScorer(D):
    tokenized_D = [tokenizer(d) for d in D]
    result = []
    for d in tokenized_D:
        result.append([(t, tfidf(t, d, tokenized_D)) for t in d])
    return result


if __name__ == '__main__':
    corpus = ['동해물과 백두산이 마르고 닳도록 하느님이 보우하사 우리나라 만세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '남산 위에 저 소나무 철갑을 두른 듯 바람서리 불변함은 우리 기상일세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '가을 하늘 공활한데 높고 구름 없이 밝은 달은 우리 가슴 일편단심일세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '이 기상과 이 맘으로 충성을 다하여 괴로우나 즐거우나 나라 사랑하세 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세',
              '오 필승 코리아 오 필승 코리아 오 필승 코리아 오 오레 오레 무궁화 삼천리 화려 강산 대한사람 대한으로 길이 보전하세']

    for i, doc in enumerate(tfidfScorer(corpus)):
        print('====== document[%d] ======' % i)
        print(doc)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk
from konlpy.tag import Hannanum

corpus = ['경구피임약을 휴약기없이 3번정도 바꿔서 복용했는데 자궁이나 신체에 문제가 일어나나요? 처음에 갈색냉에서 점점 피로 변하더니 양이 점점 많아지네요 부정출혈은 소량이라 들었는데 생리인가요? 부정출혈인가요? 아랫배도 통증이 있습니다',
          '제가 경구피임약을 한달복용하다가 좀 더 미루고 싶어서 다른 종류의 약으로 바꿔서 복용중인데요 마치 생리할때처럼 아랫배도 아프고 출혈도 좀 있네요 괜찮은건가요? 처음에 갈색냉처럼 나오다가 점점 생리처럼 출혈로 나오네요 괜찮은가요?출혈도 생리나오듯이 나와요 생리를 하는건가요? 그렇다고치기엔 양이 적네요..',
          '사후피임약을 먹고 부정출혈을 수요일부터 시작했는데 피의 양이 너무 많아서요ㅠㅠ 생리할 때 만큼 많이 나오는데 문제가 있는걸까요ㅠㅠ',
          '센스리베 복용중인데 소화불량이 너무 심해요ㅠ 밥먹으면 메스껍고 속이 아파요.. 소화도 안되고 자꾸만 토가 나옵니다ㅠ 변비와 설사도 생기구요.. 피임약을 바꾸고싶은데 꼭 한팩을 다 먹고나서 교체해야하나요? 피임약만 먹으면 너무 아파서 먹기가 싫어져요.. ㅠㅠ',
          '배란일 2일전에 관계를 질외사정으로 맺고 48시간 지나기 몇시간 전에 사후피임약 노레보를 한알 먹었어요 지금 5시간 정도 지났는데 두통이랑 피곤함이 있네요 임신 될수 있나요...ㅠㅠ',
          '1월 17일에 관계를 맺고 1월 18일에 사후피임약을 처방 받아 먹고 생리주기가 앞당겨져 23~27일까지 생리를 했습니다 그리고 2월 3일에 사후피임약을 먹었습니다. 찾아보니 같은 주기에 먹으며 효과가 없다는데 생리를 했으니 같은 주기가 아닌가요?',
          '오늘 생리 첫 날이라 경구피임약 시작하려는데 먹는 시간을 오후 2시쯤으로 하고 싶어서요 .. 첫 날 9시쯤에 먹고 그 다음날 부터 계속 같은 시간에 먹어도 될까요 ?',
          '18일에 관계를 맺고 19일에 바로 19시간만에 사후피임약을 먹었는데 생리예정일은 불규칙하지만 대충 22~23일에 해야하는데요 지금 생리가 15일째 늦어지고 있는데.. 이렇게 늦어지고 그런가요,, 불안합니다 내공100이요',
          '일주일전 관계중 콘돔이 찢어진걸 확인하고 한시간정도 후에 바로 사후피임약 복용하였습니다. 오늘 출혈이 있었는데 확실히 피임 된걸까요?.. 그리고 혹시 출혈이 있는중에 임신테스트기 사용해도 되나요??',
          '31일 오전 10시 반쯤 사후피임약 복용을 했고 오늘 아침에 생리처럼 피가 나왔는데 원래 생리주기대로면 내일이 예정일이긴 하거든요 이거 부정출혈일까요 생리일까요?',
          '경구피임약 복용하는법']

# ============================================
# -- Get TFIDF
# ============================================
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)
hannanum = Hannanum()

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(corpus):
    print('====== document[%d] ======' % i)
    print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )

In [ ]:
from google.colab import files
files.upload()


Saving sample_data_processed.xlsx to sample_data_processed.xlsx


{'sample_data_processed.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00A7\x82\xcfn\x01\x00\x00\x04\x05\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk
import pandas as pd
from konlpy.tag import Hannanum
from openpyxl import Workbook

# ============================================
# --  파일 읽어오기
# ============================================
df = pd.read_excel('/content/sample_data_processed.xlsx')
df_np = pd.DataFrame.to_numpy(df)

# ============================================
# --  형태소 분석
# ============================================
hannanum = Hannanum()

tokens = []
nouns = []
for i in range (0, len(df)) :
  tokens.append(hannanum.morphs(df_np[i][1]))
  nouns.append(hannanum.nouns(df_np[i][1]))  

# ============================================
# --  Excel 입력
# ============================================
wb = Workbook() 
sheet = wb.active 
sheet.title = '모든 품사' # 컬럼명 지정(헤더) 
col_names = ['질문', '답변'] 
for seq, name in enumerate(col_names): 
  sheet.cell(row=1, column=seq+1, value=name) 

# 시트 저장 
row_no = 1
for n, rows in enumerate(tokens): 
  for seq, value in enumerate(rows): 
    sheet.cell(row=row_no+n, column=seq+1, value=tokens[n][seq]) 

wb.save('/content/tokens.xlsx')
wb.close()

wb2 = Workbook() 
sheet = wb2.active 
sheet.title = '명사' # 컬럼명 지정(헤더) 
col_names = ['질문', '답변'] 
for seq, name in enumerate(col_names): 
  sheet.cell(row=1, column=seq+1, value=name) 

# 시트 저장 
row_no = 1
for n, rows in enumerate(nouns): 
  for seq, value in enumerate(rows): 
    sheet.cell(row=row_no+n, column=seq+1, value=nouns[n][seq]) 

wb2.save('/content/nouns.xlsx')
wb2.close()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm
from collections import defaultdict
import nltk
import pandas as pd
from konlpy.tag import Hannanum
from openpyxl import Workbook

question = []
answers = []
# ============================================
# --  파일 읽어오기
# ============================================
df = pd.read_excel('/content/sample_data_processed.xlsx')
df_np = pd.DataFrame.to_numpy(df)

for i in range(1, len(df_np)) :
  question.append(df_np[i][0])
  answers.append(df_np[i][1])

# ============================================
# -- Get TFIDF
# ============================================
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(question)
hannanum = Hannanum()

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

# for i, sent in enumerate(corpus):
#     print('====== document[%d] ======' % i)
#     print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )

# ============================================
# --  파일 입력
# ============================================
f = open('/content/TF_IDF.txt', 'w')

tmp = []

for i, sent in tqdm(enumerate(question)):
    f.write('====== %d번 ======\n' % i)
    #print([ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ])
    #f.write('[')
    for token in sent.split() :
      f.write(token+',')
      f.write(str(round(sp_matrix[i, word2id[token]], 6))+' ')
      tmp[i].append((token, round(sp_matrix[i, word2id[token]], 6)))
    f.write('\n')

f.close()

IndexError: ignored

In [ ]:
print(tmp)

[('오랫동안', 0.467846), ('피임제', 0.415668), ('복용', 0.157165), ('후,', 0.0), ('복용을', 0.231813), ('중단해도', 0.415668), ('효과가', 0.175651), ('어느', 0.354623), ('정도', 0.203505), ('유지되나요?', 0.0), ('피임제를', 0.374326), ('끊은', 0.374326), ('후', 0.0), ('얼마의', 0.400452), ('기간이', 0.400452), ('지나야', 0.400452), ('임신이', 0.279845), ('가능합니까?', 0.0), ('사전', 0.458758), ('피임제는', 0.53809), ('건강에', 0.458758), ('해롭나요?', 0.0), ('피임제가', 0.549584), ('낙태를', 0.578218), ('유발할', 0.527375), ('수', 0.0), ('있나요?', 0.0), ('피임제가', 0.492763), ('나중에', 0.442823), ('기형아를', 0.518435), ('유발할', 0.472849), ('수', 0.0), ('있나요?', 0.0), ('임신', 0.207871), ('중', 0.0), ('피임제를', 0.416313), ('실수로', 0.379707), ('복용', 0.149614), ('시', 0.0), ('태아에게', 0.445369), ('안', 0.0), ('좋은', 0.31697), ('영향을', 0.346026), ('미치나요?', 0.0), ('피임제', 0.427663), ('복용이', 0.349314), ('체중의', 0.481347), ('변화를', 0.481347), ('가져오나요?', 0.0), ('피임제', 0.395643), ('복용과', 0.379654), ('함께', 0.32316), ('흡연을', 0.445308), ('하여도', 0.445308), ('괜찮습니까?', 0.0), ('응급피임제는', 0.477465), ('여성의

In [ ]:
from openpyxl import Workbook
import pandas as pd

# ============================================
# --  파일 읽어오기
# ============================================
filePath = '/content/TF_IDF.txt'
f = open(filePath, 'r')
line = f.readlines()

each_line = []

for i in range(len(line)):
  each_word = []
  if(line[i][0] != '=') :
    each_line = line[i].split()
    print(each_line)
    for j in range()

['오랫동안,0.467846', '피임제,0.415668', '복용,0.157165', '후,,0.0', '복용을,0.231813', '중단해도,0.415668', '효과가,0.175651', '어느,0.354623', '정도,0.203505', '유지되나요?,0.0']
['피임제를,0.374326', '끊은,0.374326', '후,0.0', '얼마의,0.400452', '기간이,0.400452', '지나야,0.400452', '임신이,0.279845', '가능합니까?,0.0']
['사전,0.458758', '피임제는,0.53809', '건강에,0.458758', '해롭나요?,0.0']
['피임제가,0.549584', '낙태를,0.578218', '유발할,0.527375', '수,0.0', '있나요?,0.0']
['피임제가,0.492763', '나중에,0.442823', '기형아를,0.518435', '유발할,0.472849', '수,0.0', '있나요?,0.0']
['임신,0.207871', '중,0.0', '피임제를,0.416313', '실수로,0.379707', '복용,0.149614', '시,0.0', '태아에게,0.445369', '안,0.0', '좋은,0.31697', '영향을,0.346026', '미치나요?,0.0']
['피임제,0.427663', '복용이,0.349314', '체중의,0.481347', '변화를,0.481347', '가져오나요?,0.0']
['피임제,0.395643', '복용과,0.379654', '함께,0.32316', '흡연을,0.445308', '하여도,0.445308', '괜찮습니까?,0.0']
['응급피임제는,0.477465', '여성의,0.453822', '몸에,0.339886', '해롭지,0.51079', '않나요?,0.0']
['응급피임제는,0.575686', '병원에서,0.446935', '처방,0.324088', '받아야,0.506999', '하나요?,0.0']
['어지럼증,,0.0', '질,0.0', '출혈,